In [8]:
import sys
!{sys.executable} -m pip install geocoder
!{sys.executable} -m pip install folium
print('Package Installation Over')

Package Installation Over


In [9]:
pip install BeautifulSoup4

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd # library for data analsysis
import geocoder # import geocoder
import requests 
from bs4 import BeautifulSoup 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Pakage Import Completed..')

Pakage Import Completed..


In [3]:
print('Page ')
#URL = "https://www.wikizeroo.org/index.php?q=aHR0cHM6Ly9lbi53aWtpcGVkaWEub3JnL3dpa2kvTGlzdF9vZl9wb3N0YWxfY29kZXNfb2ZfQ2FuYWRhOl9N"
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(URL) 
  
soup = BeautifulSoup(r.content, 'html5lib') 
table = soup.find('div', attrs = {'id':'container'}) 

# print(soup.prettify()) 
print('Page Scrapped.')


Page 
Page Scrapped.


In [13]:
postalCodes = [];
boroughs= [];
neighborhoods = [];
columnNum = 1;
passVal = False

for row in soup.find_all('td'):
    for cell in row:
        if cell.string and cell.string[0].isalpha() and len(cell.string) > 2:
            passVal = False
            if columnNum == 1:
                if passVal == False and cell.string[1].isdigit():
                    postalCodes.append(cell.string);   
                    columnNum = 2
                else:
                    continue
            elif columnNum == 2 :
                if cell.string == 'Not assigned\n':
                    passVal = True
                    del postalCodes[-1]
                    columnNum = 1
                    continue
                else:
                    boroughs.append(cell.string);      
                    columnNum = 3
            elif columnNum == 3 :
                if cell.string == 'Not assigned\n':
                    neighborhoods.append(boroughs[-1])                    
                else:
                    neighborhoods.append(cell.string); 
                columnNum = 1


print('Data Collection is Over')



Data Collection is Over


In [14]:
column_names = ['PostalCode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
neighbors = pd.DataFrame(columns=column_names)
neighbors
#neighbors.shape

,PostalCode,Borough,Neighborhood,Latitude,Longitude


In [15]:
# initialize your variable to None
lat_lng_coords = None

for data in range(0, len(postalCodes)-1):
    code = postalCodes[data]
    borough = boroughs[data]
    neighborhood_name = neighborhoods[data]
    
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
    lat_lng_coords = g.latlng

    neighbors = neighbors.append({ 'PostalCode': code,
                                   'Borough': borough,
                                   'Neighborhood': neighborhood_name,
                                   'Latitude': lat_lng_coords[0],
                                   'Longitude': lat_lng_coords[1]}, ignore_index=True)
    
neighbors

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A\n,North York\n,Parkwoods\n,43.75245,-79.32991
1,M4A\n,North York\n,Victoria Village\n,43.73057,-79.31306
2,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n",43.65512,-79.36264
3,M6A\n,North York\n,"Lawrence Manor, Lawrence Heights\n",43.72327,-79.45042
4,M7A\n,Downtown Toronto\n,"Queen's Park, Ontario Provincial Government\n",43.66253,-79.39188
...,...,...,...,...,...
97,M5X\n,Downtown Toronto\n,"First Canadian Place, Underground city\n",43.64828,-79.38146
98,M8X\n,Etobicoke\n,"The Kingsway, Montgomery Road, Old Mill North\n",43.65319,-79.51113
99,M4Y\n,Downtown Toronto\n,Church and Wellesley\n,43.66659,-79.38133
100,M7Y\n,East Toronto\n,"Business reply mail Processing Centre, South C...",43.64869,-79.38544


In [16]:
neighbors.shape

(102, 5)

In [17]:
VERSION = '20180605' # Foursquare API version

neighborhood_name = neighbors.loc[0, 'Neighborhood'] # neighborhood name
neighborhood_latitude = neighbors.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighbors.loc[0, 'Longitude'] # neighborhood longitude value

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

radius = 500 # define radius
LIMIT = 100 # limit of number of venues returned by Foursquare API

# url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
#     CLIENT_ID, 
#     CLIENT_SECRET, 
#     VERSION, 
#     43.67635739999999, 
#     79.2930312, 
#     radius, 
#     LIMIT)

url = 'https://api.foursquare.com/v2/venues/explore?&client_id=ZWLCLXKRKKBH1AXN2EMN0TLRRVCPDINO1CYW4AVA5VUFANK3&client_secret=2OKBDY3EH23YLKNIS1UEDB4BHBGBLB2SP4EPOABWQ0LCJF5X&v=20180605&ll=43.67635739999999,-79.2930312&radius=500&limit=100'

results = requests.get(url).json()
results

Latitude and longitude values of Parkwoods
 are 43.75245000000007, -79.32990999999998.


{'meta': {'code': 200, 'requestId': '6034ea734c834a469ebe9f25'},
 'response': {'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 5,
  'suggestedBounds': {'ne': {'lat': 43.680857404499996,
    'lng': -79.28682091449052},
   'sw': {'lat': 43.67185739549999, 'lng': -79.29924148550948}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd461bc77b29c74a07d9282',
       'name': 'Glen Manor Ravine',
       'location': {'address': 'Glen Manor',
        'crossStreet': 'Queen St.',
        'lat': 43.67682094413784,
        'lng': -79.29394208780985,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.67682094413784,
          'lng': -79.29394208780985}],
        'distanc

In [18]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [19]:

from pandas.io.json import json_normalize

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Upper Beaches,Neighborhood,43.680563,-79.292869
4,Seaspray Restaurant,Asian Restaurant,43.678888,-79.298167


In [20]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

5 venues were returned by Foursquare.


In [21]:
def getNearbyVenues(names, latitudes, longitudes):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)


        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [22]:
toronto_venues = getNearbyVenues(names=neighborhoods,
                                 latitudes=neighbors['Latitude'],
                                 longitudes=neighbors['Longitude'])

Parkwoods

Victoria Village

Regent Park, Harbourfront

Lawrence Manor, Lawrence Heights

Queen's Park, Ontario Provincial Government

Islington Avenue, Humber Valley Village

Malvern, Rouge

Don Mills

Parkview Hill, Woodbine Gardens

Garden District, Ryerson

Glencairn

West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale

Rouge Hill, Port Union, Highland Creek

Don Mills

Woodbine Heights

St. James Town

Humewood-Cedarvale

Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood

Guildwood, Morningside, West Hill

The Beaches

Berczy Park

Caledonia-Fairbanks

Woburn

Leaside

Central Bay Street

Christie

Cedarbrae

Hillcrest Village

Bathurst Manor, Wilson Heights, Downsview North

Thorncliffe Park

Richmond, Adelaide, King

Dufferin, Dovercourt Village

Scarborough Village

Fairview, Henry Farm, Oriole

Northwood Park, York University

East Toronto, Broadview North (Old East York)

Harbourfront East, Union Station, Toronto Islands

Little Portugal, Trinit

In [23]:
print(toronto_venues.shape)
toronto_venues.head()

(510, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods\n,43.75245,-79.32991,Glen Manor Ravine,43.676821,-79.293942,Trail
1,Parkwoods\n,43.75245,-79.32991,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,Parkwoods\n,43.75245,-79.32991,Grover Pub and Grub,43.679181,-79.297215,Pub
3,Parkwoods\n,43.75245,-79.32991,Upper Beaches,43.680563,-79.292869,Neighborhood
4,Parkwoods\n,43.75245,-79.32991,Seaspray Restaurant,43.678888,-79.298167,Asian Restaurant


In [24]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt\n,5,5,5,5,5,5
"Alderwood, Long Branch\n",5,5,5,5,5,5
"Bathurst Manor, Wilson Heights, Downsview North\n",5,5,5,5,5,5
Bayview Village\n,5,5,5,5,5,5
"Bedford Park, Lawrence Manor East\n",5,5,5,5,5,5
...,...,...,...,...,...,...
"Willowdale, Willowdale West\n",5,5,5,5,5,5
Woburn\n,5,5,5,5,5,5
Woodbine Heights\n,5,5,5,5,5,5


In [25]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 5 uniques categories.


In [26]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Trail,Asian Restaurant,Health Food Store,Neighborhood,Pub
0,1,0,0,Parkwoods\n,0
1,0,0,1,Parkwoods\n,0
2,0,0,0,Parkwoods\n,1
3,0,0,0,Parkwoods\n,0
4,0,1,0,Parkwoods\n,0


In [27]:
toronto_onehot.shape

(510, 5)

In [28]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Trail,Asian Restaurant,Health Food Store,Pub
0,Agincourt\n,0.2,0.2,0.2,0.2
1,"Alderwood, Long Branch\n",0.2,0.2,0.2,0.2
2,"Bathurst Manor, Wilson Heights, Downsview North\n",0.2,0.2,0.2,0.2
3,Bayview Village\n,0.2,0.2,0.2,0.2
4,"Bedford Park, Lawrence Manor East\n",0.2,0.2,0.2,0.2
...,...,...,...,...,...
93,"Willowdale, Willowdale West\n",0.2,0.2,0.2,0.2
94,Woburn\n,0.2,0.2,0.2,0.2
95,Woodbine Heights\n,0.2,0.2,0.2,0.2
96,York Mills West\n,0.2,0.2,0.2,0.2


In [29]:
# confirm the new size

toronto_grouped.shape

(98, 5)

In [30]:
# print each neighborhood along with the top 5 most common venues

num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt
----
               venue  freq
0              Trail   0.2
1   Asian Restaurant   0.2
2  Health Food Store   0.2
3                Pub   0.2


----Alderwood, Long Branch
----
               venue  freq
0              Trail   0.2
1   Asian Restaurant   0.2
2  Health Food Store   0.2
3                Pub   0.2


----Bathurst Manor, Wilson Heights, Downsview North
----
               venue  freq
0              Trail   0.2
1   Asian Restaurant   0.2
2  Health Food Store   0.2
3                Pub   0.2


----Bayview Village
----
               venue  freq
0              Trail   0.2
1   Asian Restaurant   0.2
2  Health Food Store   0.2
3                Pub   0.2


----Bedford Park, Lawrence Manor East
----
               venue  freq
0              Trail   0.2
1   Asian Restaurant   0.2
2  Health Food Store   0.2
3                Pub   0.2


----Berczy Park
----
               venue  freq
0              Trail   0.2
1   Asian Restaurant   0.2
2  Health Food Store   0.2
3         

               venue  freq
0              Trail   0.2
1   Asian Restaurant   0.2
2  Health Food Store   0.2
3                Pub   0.2


----Parkview Hill, Woodbine Gardens
----
               venue  freq
0              Trail   0.2
1   Asian Restaurant   0.2
2  Health Food Store   0.2
3                Pub   0.2


----Parkwoods
----
               venue  freq
0              Trail   0.2
1   Asian Restaurant   0.2
2  Health Food Store   0.2
3                Pub   0.2


----Queen's Park, Ontario Provincial Government
----
               venue  freq
0              Trail   0.2
1   Asian Restaurant   0.2
2  Health Food Store   0.2
3                Pub   0.2


----Regent Park, Harbourfront
----
               venue  freq
0              Trail   0.2
1   Asian Restaurant   0.2
2  Health Food Store   0.2
3                Pub   0.2


----Richmond, Adelaide, King
----
               venue  freq
0              Trail   0.2
1   Asian Restaurant   0.2
2  Health Food Store   0.2
3                Pub   0.